### 我们定义一个函数，它能够判断两个 pattern 是不是相符

In [7]:
#检测目前这个词是否由？开头并且后面都是字母
def is_variable(pat):
    return pat.startswith('?') and all(s.isalpha() for s in pat[1:])

In [15]:
c = 'are you ?X'
is_variable(c.split()[2])

True

In [5]:
def pat_match(pattern,saying):
    if is_variable(pattern[0]): return True
    else:
        #如果模板匹配，则向后继续比较
        if pattern[0] != saying[0]: return False
        else:
            return pat_match(pattern[1:],saying[1:])

In [11]:
pat_match('I want ?X'.split(),'I want holiday'.split())

True

In [16]:
pat_match('I have dreamed a ?X'.split(), "I dreamed about dog".split())

False

In [17]:
pat_match('I dreamed about ?X'.split(), "I dreamed about dog".split())

True

### 我们想知道变量是什么

In [18]:
def pat_match(pattern,saying):
    if is_variable(pattern[0]):
        return pattern[0],saying[0]
    else:
        if pattern[0] != saying[0]: return False
        else:
            return pat_match(pattern[1:],saying[1:])

In [19]:
pattern = 'I want ?X'.split()
saying = 'I want holiday'.split()

In [20]:
pat_match(pattern,saying)

('?X', 'holiday')

In [22]:
pat_match('?X equals ?X'.split(),'2+2 equals 2+2'.split())

('?X', '2+2')

### 如果有两个变量呢？

In [25]:
def pat_match(pattern,saying):
    if not pattern or not saying: return []
    #如果前面匹配上了 就从前面开始匹配
    if is_variable(pattern[0]):
        return [(pattern[0],saying[0])] + pat_match(pattern[1:],saying[1:])
    #否则可以从后面开始匹配
    else:
        if pattern[0] != saying[0]: return []
        else:
            return pat_match(pattern[1:],saying[1:])        

In [26]:
pat_match('?X great than ?Y'.split(), '3 great than 2'.split())

[('?X', '3'), ('?Y', '2')]

### 用模板去替换变量

In [33]:
def pat_to_dict(patterns):
    return {k:v for k,v in patterns}

In [39]:
def substitute(rule,parsed_rules):
    if not rule: return []
    #rule是回应的句子和变量，parsed_rules是解析好的模板
    #回应的句子用字典的get方法输入列表，当变量和模板匹配时，放入解析好的变量
    return [parsed_rules.get(rule[0],rule[0])] + substitute(rule[1:],parsed_rules)

In [40]:
got_patterns = pat_match("I want ?X".split(), "I want iPhone".split())
substitute("What if you mean if you got a ?X".split(), pat_to_dict(got_patterns))

['What', 'if', 'you', 'mean', 'if', 'you', 'got', 'a', 'iPhone']

### 变成一句话

In [42]:
' '.join(substitute("What if you mean if you got a ?X".split(), pat_to_dict(got_patterns)))

'What if you mean if you got a iPhone'

In [43]:
john_pat = pat_match('?P needs ?X'.split(), 'John needs vacation'.split() )
' '.join(substitute("Why does ?P need ?X ?".split(), pat_to_dict(john_pat)))

'Why does John need vacation ?'

In [44]:
defined_patterns = {
    "I need ?X": ["Image you will get ?X soon", "Why do you need ?X ?"], 
    "My ?X told me something": ["Talk about more about your ?X", "How do you think about your ?X ?"]
}

In [116]:
import random
choice = random.choice

def get_response(saying):
    if saying.startswith('I'):
        x_p = pat_match(list(defined_patterns.keys())[0].split(),saying.split())
        rep = ' '.join(substitute(choice(list(defined_patterns.values())[0]).split(), pat_to_dict(x_p)))
        return rep
    elif saying.startswith('My'):
        x_p = pat_match(list(defined_patterns.keys())[1].split(),saying.split())
        rep = ' '.join(substitute(choice(list(defined_patterns.values())[1]).split(), pat_to_dict(x_p)))
        return rep
    return []

### 作业一

In [119]:
get_response('I need iphone')

'Why do you need iphone ?'

In [118]:
get_response("My mother told me something")

'Talk about more about your mother'

### 如果iphone 变成 an iphone

In [124]:
def is_pattern_segment(pattern):
    #多了一个星号(*),表示匹配多个
    return pattern.startswith('?*') and all(a.isalpha() for a in pattern[2:])

In [125]:
is_pattern_segment('?*P')

True

In [126]:
from collections import defaultdict
fail = [True, None]

#从单个词的匹配变成多个词的匹配
def pat_match_with_seg(pattern, saying):
    if not pattern or not saying: return []
    pat = pattern[0]
    #前面匹配
    if is_variable(pat):
        return [(pat, saying[0])] + pat_match_with_seg(pattern[1:], saying[1:])
    #后面匹配
    elif is_pattern_segment(pat):
        #找到匹配的内容和序号
        match, index = segment_match(pattern, saying)
        return [match] + pat_match_with_seg(pattern[1:], saying[index:])
    elif pat == saying[0]:
        return pat_match_with_seg(pattern[1:], saying[1:])
    else:
        return fail

#找到匹配内容和序号
def segment_match(pattern, saying):
    seg_pat, rest = pattern[0], pattern[1:]
    seg_pat = seg_pat.replace('?*', '?')
    if not rest: return (seg_pat, saying), len(saying)    
    #将内容和模型开始对应比较
    for i, token in enumerate(saying):
        if rest[0] == token and is_match(rest[1:], saying[(i + 1):]):
            return (seg_pat, saying[:i]), i
    return (seg_pat, saying), len(saying)

#判断是否匹配
def is_match(rest, saying):
    if not rest and not saying:
        return True
    if not all(a.isalpha() for a in rest[0]):
        return True
    if rest[0] != saying[0]:
        return False
    return is_match(rest[1:], saying[1:])

In [127]:
segment_match('?*P is very good'.split(), "My dog and my cat is very good".split())

(('?P', ['My', 'dog', 'and', 'my', 'cat']), 5)

In [128]:
pat_match_with_seg('?*P is very good and ?*X'.split(), "My dog is very good and my cat is very cute".split())

[('?P', ['My', 'dog']), ('?X', ['my', 'cat', 'is', 'very', 'cute'])]

In [129]:
response_pair = {
    'I need ?X': [
        "Why do you neeed ?X"
    ],
    "I dont like my ?X": ["What bad things did ?X do for you?"]
}

In [130]:
pat_match_with_seg('I need ?*X'.split(), 
                  "I need an iPhone".split())

[('?X', ['an', 'iPhone'])]

In [131]:
def pat_to_dict(patterns):
    return {k: ' '.join(v) if isinstance(v, list) else v for k, v in patterns}

In [132]:
substitute("Why do you neeed ?X".split(), pat_to_dict(pat_match_with_seg('I need ?*X'.split(), 
                  "I need an iPhone".split())))

['Why', 'do', 'you', 'neeed', 'an iPhone']

In [136]:
substitute("Hi, how do you do?".split(), pat_to_dict(pat_match_with_seg('?*X hello ?*Y'.split(), 
                  "I am mike, hello ".split())))

['Hi,', 'how', 'do', 'you', 'do?']

In [139]:
rules = {
    "?*X hello ?*Y": ["Hi, how do you do?"],
    "I was ?*X": ["Were you really ?X ?", "I already knew you were ?X ."]
}

In [140]:
def get_response_2(saying):
    if saying.startswith('I'):
        x_p = pat_match_with_seg(list(rules.keys())[1].split(),saying.split())
        rep = ' '.join(substitute(choice(list(rules.values())[1]).split(), pat_to_dict(x_p)))
        return rep
    else:
        x_p = pat_match_with_seg(list(rules.keys())[0].split(),saying.split())
        rep = ' '.join(substitute(choice(list(rules.values())[0]).split(), pat_to_dict(x_p)))
        return rep
    return []

### 作业一

In [152]:
get_response_2('I was happy')

'I already knew you were happy .'

In [153]:
get_response_2('hello mike')

'Hi, how do you do?'

### 尝试中文

In [185]:
import jieba

def cut(string):
    return list(jieba.cut(string))

def is_variable_c(pat):
    return pat.startswith('?')

def pat_match_c(pattern,saying):
    if not pattern or not saying: return []
    #如果前面匹配上了 就从前面开始匹配
    if is_variable_c(pattern[0]):
        return [(pattern[0],saying[0])] + pat_match_c(pattern[1:],saying[1:])
    #否则可以从后面开始匹配
    else:
        if pattern[0] != saying[0]: return []
        else:
            return pat_match_c(pattern[1:],saying[1:])  

In [186]:
xiaoming_pat = pat_match(cut('?很可爱'), cut("小明很可爱"))
''.join(substitute(cut("为什么说?很可爱"), pat_to_dict(xiaoming_pat)))

'为什么说小明很可爱'

In [206]:
defined_patterns_c = {
    "我感觉?很可爱": ["?是谁", "我不比?更可爱吗"], 
    "晚上带你去看?": ["?是啥", "看?还不如学习"]
}

In [208]:
import random
choice = random.choice

def get_response_c(saying):
    if saying.startswith('我'):
        x_p = pat_match(cut(list(defined_patterns_c.keys())[0]),cut(saying))
        rep = ''.join(substitute(cut(choice(list(defined_patterns_c.values())[0])), pat_to_dict(x_p)))
        return rep
    elif saying.startswith('晚上'):
        x_p = pat_match(cut(list(defined_patterns_c.keys())[1]),cut(saying))
        rep = ''.join(substitute(cut(choice(list(defined_patterns_c.values())[1])), pat_to_dict(x_p)))
        return rep
    return []

### 作业二

In [209]:
get_response_c('我感觉小美很可爱')

'我不比小美更可爱吗'

In [212]:
get_response_c('晚上带你去看电影')

'看电影还不如学习'